In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains

In [ ]:
driver = webdriver.Chrome();
url="https://www.bseindia.com/markets/equity/EQReports/MarketWatch.html?index_code=16"
driver.get(url)
#wait until the submit button gets rendered
submit_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.btn-default")))
ActionChains(driver).click(submit_button).perform()
#wait until the result after clicking submit_button renders
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".tdcolumn.text-left")))
page_contents=driver.page_source

In [ ]:
if "ZOMATO" not in page_contents:
    raise Exception("Page after submit not loaded")

Things that I want to extract - 
open, high, low, name

In [ ]:
soup = BeautifulSoup(page_contents, "html.parser")
bse_data = {
    "company_codes": [],
    "company_names":[],
    "opening_price":[],
    "high_price":[],
    "low_price":[],
}
rows = soup.find_all("tr", attrs={"class": "ng-scope"})

#for loop to populate bse_data
for row in rows:
    valid_tag_count=0
    for child in row.children:
        if(valid_tag_count==6):
            break
        if(child.name is not None):
            if(valid_tag_count==0):
                bse_data["company_codes"].append(child.text.strip())
            elif(valid_tag_count==1):
                bse_data["company_names"].append(child.text.strip())
            elif(valid_tag_count==3):
                bse_data["opening_price"].append(child.text.strip())
            elif(valid_tag_count==4):
                bse_data["high_price"].append(child.text.strip())
            elif(valid_tag_count==5):
                bse_data["low_price"].append(child.text.strip())
            valid_tag_count+=1

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(bse_data)
print(df)

In [ ]:
#close driver and export bse_data
driver.quit()
%store bse_data

Stored 'bse_data' (dict)
